# EDA

In [9]:
import pandas as pd

In [10]:
df = pd.read_pickle('data-ai-slop-detector/final_detection.pkl')

In [15]:
df

,commenter_id,comment_id,parent_id,post_id,comment_content,cleaned_content,num_emojis,num_text_emojis,num_caps_words,num_unicode_chars,...,lsa_dim_54,lsa_dim_55,lsa_dim_56,lsa_dim_57,lsa_dim_58,lsa_dim_59,lsa_dim_60,lsa_dim_61,label,ai_confidence
0,AdamParkhomenko,1,NaN,1,https://t.co/rAkU7CWOVE,link,0,0,0,0,...,-0.000344,-0.000402,-0.000916,0.000051,0.000233,0.000026,0.000140,0.000635,LABEL_1,0.507899
1,SusanSaoirse,2,1.0,1,"@AdamParkhomenko Thing is, a good number of ma...",thing good number maga use medicaid oregon med...,0,0,0,0,...,-0.009026,0.020254,-0.004191,0.003561,0.007515,-0.014450,-0.000272,-0.019075,LABEL_0,0.033355
2,RealStarTrump,3,1.0,1,@AdamParkhomenko You’re a lying bastard. https...,youre lying bastard link,0,0,0,1,...,0.006115,0.010444,-0.002105,-0.013546,0.006378,0.002457,-0.002760,0.000702,LABEL_1,0.917006
3,catothewis13876,4,1.0,1,@AdamParkhomenko The false premise is that ill...,false premise illegals get less than american ...,0,0,0,0,...,-0.055354,-0.008987,0.013624,-0.005087,-0.059042,-0.005017,-0.034343,-0.005884,LABEL_1,0.998461
4,masterson11776,5,1.0,1,@AdamParkhomenko Emergency rooms in So Cal are...,emergency rooms indiana so cal full illegal br...,0,0,0,1,...,-0.003320,-0.004874,0.007576,-0.008425,-0.007787,-0.004375,-0.002130,-0.005013,LABEL_0,0.115065
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139402,ColborneGreg,139403,NaN,44,@nvidia @ylecun @soumithchintala @Meta @nyuniv...,i just received push notification nvidia adver...,0,0,2,0,...,-0.032044,0.004067,-0.016273,-0.022032,-0.014603,-0.027949,0.053836,-0.015425,LABEL_1,0.990853
139403,trust_Mina,139404,NaN,44,@nvidia @ylecun @soumithchintala @Meta @nyuniv...,wow seeing indiana nyc makes maine feel so luc...,1,0,3,2,...,-0.065151,-0.042522,0.104675,0.088470,0.076652,0.033047,0.095929,-0.017154,LABEL_1,0.999592
139404,damnThoughtful,139405,NaN,44,@nvidia @ylecun @soumithchintala @Meta @nyuniv...,useless gimped bandwidth buy gpu instead,0,0,1,0,...,0.003692,-0.000820,-0.004070,0.000330,0.001171,0.001398,0.000775,0.000464,LABEL_0,0.401554
139405,honasu,139406,NaN,44,@nvidia @ylecun @soumithchintala @Meta @nyuniv...,nvidia marketing play horrible part positionin...,0,0,2,0,...,-0.013342,0.032919,0.047874,0.043419,-0.010128,0.005432,-0.000098,0.020415,LABEL_0,0.001700
